In [ ]:
### 해당 코드는 모든 인공지능 코드의 뼈대이자 핵심이다.
### 이것을 잘 마스터해두면 Segmentation 코드도 이해 가능하다. 

In [ ]:
# Mount Google Drive Folder (/content/drive/My Drive/Colab)
# 1.Colab에서 구글 드라이버를 사용하기 위해 import 하는 부분
from google.colab import drive

# 2.drive를 colab으로 mount해온다. drive의 하위로 위치가 잡혀야 한다. 
drive.mount('/content/drive') 

# import user defined files 
import sys
import_dir ="/content/drive/My Drive/Colab Notebooks/2. Classification/pytorch-cifar"
sys.path.insert(0, import_dir)

# import ipynb files...
!pip install import_ipynb 
import import_ipynb

%run '/content/drive/My Drive/Colab Notebooks/2. Classification/pytorch-cifar/dataset.ipynb'

import easydict 
# 파이썬에서 실행 시 값 입력 python main.py --lr 0.01 -- resume False.. 이런식으로 들어간다
# ipynb은 라인기반으로 작동하므로 위와 같이 값을 넣을 수 없음.. 이를 위해 easydict 사용
# argparse.. 를 colab에서 사용 불가. easydict는 args형태로 변경해주는 역할
args = easydict.EasyDict({ "lr": 0.1, "resume": False}) 

# Debugging Tool
import pdb

# Download & Unzip Dataset (Move from google drive to local)
# 3.Google Drive에 있는 cifar.tgz 압축파일을 복사해 colab의 로컬(content/sample_data) 이 안으로 저장해오겠다.
# 절대경로로 잡음. /content/drive/ 아래는 모두 구글 drive 부분
%cp -r '/content/drive/My Drive/Colab Notebooks/Dataset/CIFAR10/cifar.tgz' '/content/sample_data'

# 4. colab의 로컬 위치에서 (content/sample_data) 이 안에서 압축을 풀었다.
!tar -xvf  '/content/sample_data/cifar.tgz'

# Initilize Additional parameters
train_root = "/content/cifar/train"
test_root = "/content/cifar/test"

In [ ]:
'''Train CIFAR10 with PyTorch.'''
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms

import os
import argparse

'''
from models import * 
models 패키지에 있는 모든 것을 다 가져와라... *의 의미
models 안으로 들어가면 여러개의 파일이 있다.
1. __init__.py 를 제일 먼저 찾게 되어있다. 가장 먼저 import 된다.
2. __init__.py 이 안으로 들어가면
    우리가 선택해 사용할 수 있는 여러 모델들이 나와있다.
    모델들이 import 되어진다.

결론은 __init__이 먼저 import 되고 __init__에 들어있는 것들이 나중에 import되어지는
이중적인 구조로 되어있다.
'''
from models import * # models의 모든 것을 호출하여 사용
from utils import progress_bar # utils를 호출하여 사용

'''
# Removed Code (Because ipynb file does not support argparse)
# Even removed, these lines are substitued by easydict code

parser = argparse.ArgumentParser(description='PyTorch CIFAR10 Training')
parser.add_argument('--lr', default=0.1, type=float, help='learning rate')
parser.add_argument('--resume', '-r', action='store_true',
                    help='resume from checkpoint')
args = parser.parse_args()
'''

device = 'cuda' if torch.cuda.is_available() else 'cpu' # 사용 device가 gpu인지 cpu인지 확인
best_acc = 0  # best test accuracy.. 체크포인트보다 더 클 경우 받아올 변수
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

# Data
print('==> Preparing data..')

#데이터 변형.. 전처리 과정..Data Augmentation
transform_train = transforms.Compose([ 
    # 패딩으로 40*40가 된 이미지를 랜덤으로 잘라 사이즈 크기(32)로 출력.. 
    # padding을 주고 crop이 될 경우 padding이 적용된 부분은 0으로 보인다. 따라서 darkness, crop 둘 다 적용
    transforms.RandomCrop(32, padding=4), 
    transforms.RandomHorizontalFlip(), # 이미지를 랜덤으로 수평으로 뒤집는다.(좌우반전)
    transforms.ToTensor() # Tensor형으로 전환
])

transform_test = transforms.Compose([
    transforms.ToTensor() # 테스트데이터를 Tensor형으로 전환
])

# Modified for custom dataset
#trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
# dataset.ipynb에서 호출된다.
trainset = Dataset(root=train_root, transforms=transform_train) # 직접경로를 설정하여 train data를 transform형식으로 불러온다...로컬에 있는 것을 받아옴.. 튜닝할 때 사용
trainloader = torch.utils.data.DataLoader( # 배치사이즈 형태로 만들어서 실제로 학습할때 이용할 수 있는 형태로 만든다.. 128개로 쪼개서 data load시마다 load됨.
    trainset, batch_size=128, shuffle=True, num_workers=os.cpu_count(), drop_last=True) # drop_last는 기본이 False,, 실제로는 거의 True. 나머지는 버리겠다.

# Modified for custom dataset
#testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testset = Dataset(root=test_root, transforms=transform_test) # 직접경로를 설정하여 test data를 transform형식으로 불러온다...튜닝할 때 사용
testloader = torch.utils.data.DataLoader( # 배치사이즈 형태로 만들어서 실제로 학습할때 이용할 수 있는 형태로 만든다.
    testset, batch_size=100, shuffle=False, num_workers=os.cpu_count(), drop_last=True)

classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck') # labels

print('==> Building model..')
# net = VGG('VGG19')
# net = ResNet18()
# net = PreActResNet18()
# net = GoogLeNet()
# net = DenseNet121()
# net = ResNeXt29_2x64d()
net = MobileNet() # 가장 가벼운 모델
# net = MobileNetV2()
# net = DPN92()
# net = ShuffleNetG2()
# net = SENet18()
# net = ShuffleNetV2(1)
# net = EfficientNetB0()
# net = RegNetX_200MF()


net = net.to(device)

# 우리는 gpu서버를 한대 돌리므로 필요 없지만.. 여러대 돌릴 경우 아래 줄 필수이다!
if device == 'cuda':
    net = torch.nn.DataParallel(net) # DataParallel..모델을 병렬로 실행하여 다수의 gpu에서 사용 가능하다.
    cudnn.benchmark = True # 내장된 cudnn 자동 튜너를 활성화하여, 하드웨어에 맞게 사용할 최상의 알고리즘을 찾는다.
'''
가장 좋은 결과를 저장하여 불가피하게 중지되었을 경우 처음부터 학습이 진행되는 것이 아니라
가장 좋은 결과가 도출되었을 때부터 다시 학습시킨다. 이를 위해 true로 바꾸어 학습하면
가장 좋은 결과가 나왔을 때 부터 학습이 이어서 시작된다.

처음에는 결과가 존재하지 않으므로 false로 두어야 오류가 나지 않는다.
'''
if args.resume: #처음에 false로 지정되어있다.. 그 후 true로 바꾸어 돌린다..
    # Load checkpoint.
    print('==> Resuming from checkpoint..')   # 데이터를 효율적으로 실행시키기위해 checkpoint를 지정해 중간중간 저장..
    assert os.path.isdir('checkpoint'), 'Error: no checkpoint directory found!'  # assert.. 디버깅.. checkpoint가 없으면 error 띄운다
    checkpoint = torch.load('./checkpoint/ckpt.pth') # 저장된 객체 파일들을 역직렬화하여 메모리에 올립니다. 
    
    net.load_state_dict(checkpoint['net']) # 역직렬화된 state_dict 를 사용하여 모델의 매개변수들을 불러옵니다
    best_acc = checkpoint['acc'] # checkpoint에 저장된 best_acc를 받아온다.
    start_epoch = checkpoint['epoch'] # checkpoint에 저장된 start_epoch을 받아온다.

criterion = nn.CrossEntropyLoss() # 손실함수 crossentropy 사용
optimizer = optim.SGD(net.parameters(), lr=args.lr,
                      momentum=0.9, weight_decay=5e-4) # SGD사용


# Training
def train(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0    

'''
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        trainloader가 5만개라면 image label이 100개씩 압축되어져 할당
        이는 곧 100번 호출되었다는 얘기와 같다. 호출 시 dataloader가 해주는 역할

'''
    for batch_idx, (inputs, targets) in enumerate(trainloader): # trainloader의 데이터를 enumerate함수를 활용해 인덱스를 지정
        inputs, targets = inputs.to(device), targets.to(device) # input과 target을 gpu로 사용할 수 있도록
        optimizer.zero_grad() # 처음 시작을 0으로 하기위해
        outputs = net(inputs) # 사용하는 모델에 inputs를 넣어 outputs 도출
        loss = criterion(outputs, targets) # 도출된 outputs를 손실함수에 적용하여 target과 비교 후, loss값 도출
        loss.backward() # loss 값에 대한 변화도를 전달받고 모델의 매개변수들에 대한 손실의 변화도 계산
        optimizer.step() #step 함수를 호출하면 매개변수 갱신
        train_loss += loss.item() # loss가 가진 값이 스칼라 형태이므로 item()를 사용해 train_loss에 더해준다.
        _, predicted = outputs.max(1) # outputs를 max로 정렬해 첫 번째 즉, 가장 큰 값을 predicted에 할당   
        total += targets.size(0) # target의 전체 개수를 total에 더해준다.
        correct += predicted.eq(targets).sum().item() # 예측값이 target값과 같을 경우 sum()으로 합쳐 correct에 더해줌.

        progress_bar(batch_idx, len(trainloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)' 
                     % (train_loss/(batch_idx+1), 100.*correct/total, correct, total)) # 결과값 출력
        #print(batch_idx,'/', len(trainloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
        #             % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))


def test(epoch):  # validation test../ 최저의 로스만 저장한다. 가장 최적만을 저장. 나머지는 저장하지 않음
    global best_acc #전역변수로 지정
    net.eval() 
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            # 딥러닝은 후처리도 중요하다.. 따라서 출력이 잘 되어야할 필요도 존재
            # progress_bar.. utils에서 불러서 사용
            progress_bar(batch_idx, len(testloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                         % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))
            #print(batch_idx,'/', len(testloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
            #             % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))

    # Save checkpoint.   
    acc = 100.*correct/total
    if acc > best_acc: # acc가 best_acc보다 크다면
        print('Saving..')
        state = {
            'net': net.state_dict(), # 처음 돌 때는 무조건 weight 값 저장
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('checkpoint'):
            os.mkdir('checkpoint') # 없으면 directory 생성
        torch.save(state, './checkpoint/ckpt.pth') # 체크포인트를 저장
        best_acc = acc



# 호출이 여기서 일어난다..
for epoch in range(start_epoch, start_epoch+20):
    train(epoch) # train먼저 돌고
    test(epoch) #test가 돌고 test function 내부에서 savepoint가 실행된다. 